# 程式作業2
* flag variable: call or put(0 or 1)

In [1]:
import numpy as np
from scipy.stats import norm
from numpy import *
from pandas.core.frame import DataFrame

In [11]:
# 期數
n = 500

#parameter
s0 = 50
K = 50
numsim = 10000
numrep = 20
T = 0.5
r = 0.1
q = 0.05
sigma = 0.4
dt = T/n
u = np.exp(sigma*sqrt(dt))
d = np.exp(-sigma*sqrt(dt))
p =  (np.exp((r-q)*dt)-d) / (u-d)


# CRR model建置
## 由2維陣列紀錄各期資訊
* 將二元樹的圖 轉換至 對角線以上(上三角)的位置
* column: 越大 期數越大
* row: 越小 股價上漲次數越多

In [28]:
def CRRmodel(s0,u,d,p,dt,optiontype): # optiontype 1:EU 0:AM    
    call = zeros((n+1,n+1))  #陣列從零開始
    put = zeros((n+1,n+1))
    region = ""
    
    for j_inv in range(0,n+1):  
        j = n - j_inv  # 期數由最後一期往前推
        for i in range(0,j+1): # 計算到對角線的row
            if j == n: 
                call[i][n] = max( s0*power(u,n-i)*power(d,i)-K , 0)
                put[i][n] = max( K-s0*power(u,n-i)*power(d,i) , 0)
            else:
                callvalue = np.exp(-r*dt)*(p*call[i][j+1] + (1-p)*call[i+1][j+1])
                putvalue = np.exp(-r*dt)*(p*put[i][j+1] + (1-p)*put[i+1][j+1])
                if optiontype == 1: #Europe
                    call[i][j] = callvalue
                    put[i][j] = putvalue
                    region = "歐式"
                elif optiontype == 0: #American
                    call[i][j] = max(callvalue , s0*power(u,j-i)*power(d,i)-K )
                    put[i][j] = max(putvalue , K-s0*power(u,j-i)*power(d,i) )
                    region = "美式"
    print("CRR model:")
    print("%s call: %f\n%s  put: %f"%(region,call[0][0],region,put[0][0]))                                    
    #return (call,put)


In [29]:
%%time
CRRmodel(s0,u,d,p,dt,1)
CRRmodel(s0,u,d,p,dt,0)

CRR model:
歐式 call: 6.036903
歐式  put: 4.832879
CRR model:
美式 call: 6.037103
美式  put: 4.985903
Wall time: 4.41 s


In [5]:
def bsprice(s0,r,q,K,T,sigma):
    v1 = (np.log(s0/K)+(r-q + sigma**2 /2)*T ) / (sigma*np.sqrt(T))
    v2 = (np.log(s0/K)+(r-q - sigma**2 /2)*T ) / (sigma*np.sqrt(T))
    call = s0*np.exp(-q*T)*norm.cdf(v1) - np.exp(-r*T)*K*norm.cdf(v2)
    put = call - s0*np.exp(-q*T) + K*np.exp(-r*T)
    print("Black-Scholes:")
    print("call: %f\n put: %f"%(call,put))  
    
    #return (call,put)

In [6]:
bsprice(s0,r,q,K,T,sigma)

Black-Scholes:
call: 6.039621
 put: 4.835596


In [7]:
def MCsimulation(s0,r,q,K,T,sigma,numrep,numsim):
    a = []
    b = []
    for i in range(1,numrep+1):
        lnst = np.random.normal(loc=np.log(s0)+(r-q - sigma**2 /2)*T, scale=sigma*np.sqrt(T), size=numsim)
        stock = np.exp(lnst)
        callpayoff = [ np.exp(-r*T) * max(0,st-K) for st in stock]
        putpayoff = [ np.exp(-r*T) * max(0,K-st) for st in stock]
        a.append(np.mean(callpayoff))  
        b.append(np.mean(putpayoff))
    callmn = np.mean(a)
    callsd = np.std(a)
    putmn = np.mean(b)
    putsd = np.std(b)
    print ('Monte Carlo simulation:')
    print ('call的95%%信賴區間為=(%f,%f)' %(callmn-2*callsd,callmn+2*callsd))
    print (' put的95%%信賴區間為=(%f,%f)' %(putmn-2*putsd,putmn+2*putsd))
    
MCsimulation(s0,r,q,K,T,sigma,numrep,numsim)

Monte Carlo simulation:
call的95%信賴區間為=(5.894481,6.210382)
 put的95%信賴區間為=(4.708346,4.947850)


# Bonus1
* CRR with one column vector

In [8]:
def CRRmodel_onevector(s0,u,d,p,dt,optiontype): # optiontype 1:EU 0:AM    
    call = [[]]*(n+1)  #陣列從零開始
    put = [[]]*(n+1)
    region = ""
    for j in range(0,n+1):
        for i in range(0,n-j+1): 
            if j == 0:
                call[i] = max( s0*power(u,n-i)*power(d,i)-K , 0)
                put[i] = max( K-s0*power(u,n-i)*power(d,i) , 0)
            else:
                callvalue = np.exp(-r*dt)*(p*call[i] + (1-p)*call[i+1])
                putvalue = np.exp(-r*dt)*(p*put[i] + (1-p)*put[i+1])
                if optiontype == 1: #Europe
                    call[i] = callvalue
                    put[i] = putvalue
                    region = "歐式"
                elif optiontype == 0: #American
                    call[i] = max(callvalue , s0*power(u,n-j-i)*power(d,i)-K )
                    put[i] = max(putvalue , K-s0*power(u,n-j-i)*power(d,i) )
                    region = "美式"
    print("CRR model_vector one:")
    print("%s call: %f\n%s  put: %f"%(region,call[0],region,put[0]))                                    
    #return (call,put)


In [9]:
CRRmodel_onevector(s0,u,d,p,dt,1)

CRR model_vector one:
歐式 call: 6.036903
歐式  put: 4.832879


# Bonus2: Combinatorial method

In [16]:
def combmethod(s0,u,d,p):
    count = 0
    test = 0
    for j in range(0,n+1):   
        if s0*(u**(n-j))*(d**j) > K:
            if j==0:
                logcomb = 0
            else: 
                logcomb = sum(np.log([n-i for i in range(j)])) - sum(np.log([i+1 for i in range(j)]) )
            mid = (n-j)*np.log(p) + j*np.log(1-p)
            count = count + np.exp(logcomb + mid) * (s0*(u**(n-j))*(d**j)-K)         
    count = count*np.exp(-r*T) 
    print('Combinatorial method:\ncall: %f'%count)
combmethod(s0,u,d,p)                                                            

Combinatorial method:
call: 6.036903


# 硬幹的Combinational method

In [11]:

from scipy.special import comb, perm

count = 0
for j in range(0,n+1): 
    if s0*(u**(n-j))*(d**j) > K:
        count = count + comb(n,j)*(p**(n-j))*((1-p)**j) * (s0*(u**(n-j))*(d**j)-K)
count = count *np.exp(-r*T)
count

6.036903158041655